<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Metrices/Descriptive_Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation

Cloning into 'Objective-criterias-to-quantify-the-accuracy-of-explanation'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 164 (delta 29), reused 0 (delta 0), pack-reused 101
Receiving objects: 100% (164/164), 8.54 MiB | 3.41 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install shap
!pip install lime
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 10.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283859 sha256=91bea2433ec764671e9bd4fe69b6c321f8c727f65ece32fc93eff28e43c303fa
  Stored in directory: /root/.cache/pip/wheels/ed/d7/c9/5a0130d06d6310bc6cbe55220e6e72dcb8c4eff9a478717066
Successfully built lime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 26.

In [4]:
import torch
import pandas as pd
import numpy as np
import shap
from transformers import BertTokenizer
import scipy as sp
import torch
import lime
import tensorflow as tf
from lime.lime_text import LimeTextExplainer
import transformers
import torch.nn as nn

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)    

In [7]:
df = pd.read_csv('/content/Objective-criterias-to-quantify-the-accuracy-of-explanation/Dataset/test .csv')
df.head()

,hindi_text,label
0,मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा ह...,0
1,मैं किशोरावस्था से ही गंभीर अवसाद के दौर से गु...,1
2,मुझे शौक के तौर पर खाना बनाना पसंद है,0
3,"मैं इन दिनों बहुत कम महसूस कर रहा हूं, ऐसा महस...",1
4,हाल ही में ब्रिटेन की महारानी एलिजाबेथ का निधन...,2


In [8]:
text = df['hindi_text'][0]

In [9]:
test_data = df['hindi_text'][:10]
test_label = df['label'][:10]

In [10]:
# Define a function to preprocess the text data for the MuRIL model

def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()
      # predictions = tf.nn.softmax(logits)
      # prediction_label = class_name[np.argmax(predictions[0])]
  
  return scores


In [11]:
## generate explanation with LIME

class_name = ['Introduction','Depression','Grey Area']
explainer = LimeTextExplainer(class_names = class_name)

lime_explanations = []
for i in range(len(test_data)):
    text = test_data.iloc[i]
    label = test_label.iloc[i]
    explanation = explainer.explain_instance(text,adapter)
    exp = explanation.as_list()
    lime_explanations.append(exp)

In [12]:
## SHAP adaptor and predictor
def adapter_shap(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  return class_name

def predictor(x):
    probas = model(**tokenizer(x, return_tensors="pt", padding=True))[0].softmax(1).detach().cpu().numpy()
    # logits = model(**tokenizer(x, return_tensors="pt", padding=True)).logits
    # predicted_class_id = logits.argmax().item()
    # classifications = ['Introduction','Depression','Grey Area']
    # classification = logits.argmax().item()
    # class_name = classifications[classification]
    # print(class_name)
    s = sp.special.logit(probas[:,1])
    return s

def f_batch(x):
    val = np.array([])
    for i in x:
      val = np.append(val, predictor(i))
    return val

In [13]:
explainer = shap.Explainer(f_batch, tokenizer)
def explain(data):
  test = { 'text': [data]}
  shap_values = explainer(test)
  values = shap_values[0].values
  return values


In [14]:
# Generate SHAP explanations for the test data
shap_explanations = []
for i in range(len(test_data)):
    text = test_data.iloc[i]
    exp = explain(text)
    shap_explanations.append(exp)

  0%|          | 0/498 [00:00<?, ?it/s]

In [15]:
shap_explanations

[array([ 0.        , -0.17584711, -0.07410901,  0.00491303,  0.00748515,
        -0.17869902,  0.05388571, -0.23271079,  0.1354323 , -0.06935882,
        -0.04176891,  0.16749254, -0.19263017,  0.10415689,  0.03797076,
        -0.00252977,  0.29511051,  0.        ]),
 array([ 0.        , -0.04827008,  0.01906296,  0.02524168,  0.30950941,
         1.44818676,  2.0340337 ,  0.14663959,  0.05116355, -0.00220032,
         0.0500427 ,  0.03495998, -0.1944682 ,  0.        ]),
 array([ 0.        ,  0.0316658 , -0.03452772, -0.02391022,  0.06146817,
         0.00067987, -0.07124786, -0.07557167,  0.17488965, -0.0861263 ,
         0.        ]),
 array([ 0.        ,  0.04489193, -0.07331972,  0.11212334,  0.07395176,
        -0.13861029,  1.35944595,  0.14788711,  0.06811236, -0.08304728,
         0.03909224,  0.20768119,  0.85287174,  0.03398972,  0.2667477 ,
         0.02917912,  0.35837023,  0.27830581,  0.26459849, -0.05213336,
        -0.06430765,  0.00701184, -0.04435872, -0.04086093,  0.

In [16]:
lime_explanations

[[('म', -0.0370713972147079),
  ('द', -0.028765689671544307),
  ('बढ', 0.014092375755308759),
  ('थ', -0.00912996492956747),
  ('पल', 0.008977791628462815),
  ('र', -0.006395887535333483),
  ('प', -0.005665860222769039),
  ('ई', 0.0048304057531917835),
  ('क', 0.004773072454394826),
  ('ह', 0.003009226927312217)],
 [('अवस', 0.25216852218171826),
  ('द', 0.12367099672042817),
  ('ग', 0.11048125331100984),
  ('स', 0.10852881106574269),
  ('भ', 0.10743760595482073),
  ('ह', 0.09643539876762763),
  ('क', 0.08485696996709649),
  ('र', 0.07861539178138527),
  ('रह', 0.05316475252928799),
  ('म', 0.015000493239173764)],
 [('बन', -0.010469351020802601),
  ('न', -0.008606289160093249),
  ('ह', -0.0069268390347600045),
  ('श', -0.006281769209431575),
  ('ख', -0.005274088652315561),
  ('म', -0.004646250443314208),
  ('झ', -0.0028586307081344803),
  ('क', -0.002728817185332599),
  ('र', -0.002717016604245387),
  ('पस', -0.0015631924331154924)],
 [('महस', 0.3478822141847646),
  ('ज', 0.084460843571

In [22]:
lime_scores = []
for instance in lime_explanations:
  individual_score = []
  for tuple in instance:
    feature, score = tuple
    individual_score.append(score)
  lime_scores.append(individual_score)

In [40]:
shap_scores = []
for instance in shap_explanations:
  individual_score = instance.tolist()
  shap_scores.append(individual_score)

In [41]:
shap_scores

[[0.0,
  -0.1758471131324768,
  -0.07410901412367821,
  0.004913028329610825,
  0.007485151290893555,
  -0.17869901657104492,
  0.05388571321964264,
  -0.2327107936143875,
  0.13543229550123215,
  -0.06935881823301315,
  -0.04176890850067139,
  0.16749253869056702,
  -0.19263016991317272,
  0.10415688715875149,
  0.03797076269984245,
  -0.002529773861169815,
  0.2951105087995529,
  0.0],
 [0.0,
  -0.048270076513290405,
  0.019062958657741547,
  0.025241680443286896,
  0.3095094133168459,
  1.448186757043004,
  2.034033696167171,
  0.1466395864263177,
  0.051163554191589355,
  -0.0022003203630447388,
  0.05004269629716873,
  0.034959979355335236,
  -0.1944682002067566,
  0.0],
 [0.0,
  0.031665802001953125,
  -0.034527719020843506,
  -0.023910224437713623,
  0.06146816909313202,
  0.0006798654794692993,
  -0.07124786078929901,
  -0.07557167112827301,
  0.17488965392112732,
  -0.08612629771232605,
  0.0],
 [0.0,
  0.04489193111658096,
  -0.0733197215013206,
  0.11212333617731929,
  0.073

In [49]:
lime_scores

[[-0.0370713972147079,
  -0.028765689671544307,
  0.014092375755308759,
  -0.00912996492956747,
  0.008977791628462815,
  -0.006395887535333483,
  -0.005665860222769039,
  0.0048304057531917835,
  0.004773072454394826,
  0.003009226927312217],
 [0.25216852218171826,
  0.12367099672042817,
  0.11048125331100984,
  0.10852881106574269,
  0.10743760595482073,
  0.09643539876762763,
  0.08485696996709649,
  0.07861539178138527,
  0.05316475252928799,
  0.015000493239173764],
 [-0.010469351020802601,
  -0.008606289160093249,
  -0.0069268390347600045,
  -0.006281769209431575,
  -0.005274088652315561,
  -0.004646250443314208,
  -0.0028586307081344803,
  -0.002728817185332599,
  -0.002717016604245387,
  -0.0015631924331154924],
 [0.3478822141847646,
  0.0844608435718606,
  0.07428446170342137,
  0.07316835581423602,
  0.06668585589933634,
  0.05966628974370376,
  0.048003459189821164,
  0.04491630334978854,
  0.03079431527803788,
  -0.027955329140204314],
 [-0.08345370990043886,
  0.0817462549

In [50]:
def descriptive_accuracy_lime(lime_scores, threshold):
    """
    Computes the descriptive accuracy of LIME explanations given a threshold.
    """
    num_instances = len(lime_scores)
    print(num_instances)
    lime_correct = np.sum(np.abs(lime_scores) >= threshold)
    lime_coverage = lime_correct / num_instances
    lime_acc = lime_correct / np.sum(np.abs(lime_scores))
    
    return lime_acc, lime_coverage

In [51]:
# Assuming that you have the LIME scores in a numpy array, lime_scores
threshold = 0.1
lime_acc, lime_coverage = descriptive_accuracy_lime(lime_scores, threshold)
print(f"LIME accuracy: {lime_acc:.3f}, LIME coverage: {lime_coverage:.3f}")

10
LIME accuracy: 3.364, LIME coverage: 1.800


In [54]:
print(type(lime_scores[0]))

<class 'list'>


In [59]:
for i in shap_scores:
  print(len(i))

18
14
11
25
13
14
15
14
12
13


In [57]:
np.abs(shap_scores) >= 0.2

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


TypeError: ignored

In [74]:
def descriptive_accuracy_shap(shap_scores, true_labels):
    """
    Computes the descriptive accuracy of LIME and SHAP explanations given a threshold.
    """
    max_features = max([len(score) for score in shap_scores])
    
    # Pad the SHAP scores with zeros to make them equal in length
    shap_scores_padded = []
    for score in shap_scores:
        if len(score) < max_features:
            score = np.pad(score, (0, max_features - len(score)), mode='constant')
        shap_scores_padded.append(score)
    
    # Convert the padded SHAP scores to a numpy array
    shap_scores_padded = np.array(shap_scores_padded)
    
    # Calculate the descriptive accuracy
    sorted_indices = np.argsort(-np.abs(shap_scores_padded), axis=1)
    top_features = sorted_indices[:, :int(np.ceil(np.mean(true_labels)))]
    accuracy = np.mean([1 if i in top_features[j] else 0 for j, i in enumerate(np.where(true_labels == 1)[0])])
    
    return accuracy

In [75]:
# Assuming that you have the SHAP scores in a numpy array, lime_scores
threshold = 0.1
shap_acc, shap_coverage = descriptive_accuracy_shap(shap_explanations,test_label)
print(f"LIME accuracy: {shap_acc:.3f}, LIME coverage: {shap_coverage:.3f}")

TypeError: ignored

In [76]:
def get_lime_descriptive_accuracy(lime_score, true_labels):
    """
    Calculates the descriptive accuracy of LIME explanations for a given set of true labels.

    Parameters:
    lime_score (list or numpy array): A list of tuples or a numpy array of feature weights
    true_labels (array): An array of true labels for each instance

    Returns:
    float: The descriptive accuracy of LIME explanations
    """
    # Convert lime_score to a list of tuples if it is a numpy array
    if type(lime_score) == np.ndarray:
        lime_score = [(i, lime_score[i]) for i in range(len(lime_score))]

    # Calculate the descriptive accuracy
    sorted_indices = np.argsort(-np.abs([score[1] for score in lime_score]))
    top_features = sorted_indices[:int(np.ceil(np.mean(true_labels)))]
    accuracy = np.mean([1 if i in top_features else 0 for i in np.where(true_labels == 1)[0]])

    return accuracy

In [78]:
l= get_lime_descriptive_accuracy(lime_scores,test_label)

In [79]:
l

0.25

In [80]:
def get_shap_descriptive_accuracy(shap_scores, true_labels):
    """
    Calculates the descriptive accuracy of SHAP explanations for a given set of true labels.

    Parameters:
    shap_scores (list of numpy arrays): A list of SHAP scores for each instance
    true_labels (array): An array of true labels for each instance

    Returns:
    float: The descriptive accuracy of SHAP explanations
    """
    # Compute the average absolute SHAP value for each feature across all instances
    num_instances = len(shap_scores)
    num_features = shap_scores[0].shape[0]
    feature_values = np.zeros((num_features,))
    for i in range(num_instances):
        instance_values = np.abs(shap_scores[i]).mean(axis=0)
        feature_values += instance_values
    feature_values /= num_instances

    # Select the top features based on their average absolute SHAP value
    sorted_indices = np.argsort(-feature_values)
    top_features = sorted_indices[:int(np.ceil(np.mean(true_labels)))]

    # Calculate the descriptive accuracy
    accuracy = np.mean([1 if i in top_features else 0 for i in np.where(true_labels == 1)[0]])

    return accuracy

In [83]:
p = get_shap_descriptive_accuracy(shap_explanations, test_label)

In [84]:
p

0.0

In [ ]:
"""

The updated get_shap_descriptive_accuracy() function calculates the descriptive accuracy of SHAP explanations
 in a similar way as the LIME case. The descriptive accuracy is defined as the percentage of instances where 
 the top features selected by SHAP explanations contain the true features that contribute to the model's prediction.

Specifically, the function first computes the average absolute SHAP value for each feature across all instances,
 and then selects the top features based on this average value. The number of top features is still determined 
 by the average number of positive labels, as in the LIME case.

Then, for each instance with a true positive label, the function checks if the true features are among the top selected features, 
and computes the percentage of such instances. This percentage is the descriptive accuracy of SHAP explanations.

"""